Assignment 2

# Langchain QnA 

* Data source: Arxiv.org/physics - data have been embedded into Azure AI Search (Azure Cognitive Search)


#### Setup Azure OpenAI

In [6]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT","").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

#### Setup the Langchain Azure Cognitive Search Retriever

* https://python.langchain.com/docs/integrations/retrievers/azure_cognitive_search#using-the-azure-cognitive-search-retriever

In [7]:
from langchain.retrievers import AzureCognitiveSearchRetriever

# os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "<YOUR_ACS_SERVICE_NAME>"
# This is NOT the whole end point string, like: https://yongaisearch.search.windows.net
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "yongaisearch"

#  os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "<YOUR_ACS_INDEX_NAME>"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "langchain-vector-arxiv-physics"

# os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = os.getenv("AAG_AZURE_SEARCH_ADMIN_KEY")

# retriever = AzureCognitiveSearchRetriever(content_key="content", top_k=3)

# retriever.get_relevant_documents("what is Oumuamua?")

#### Use langchain: load_summarize_chain to summarize document(s)

In [8]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import AzureChatOpenAI

# Construct Azure OpenAI
llm = AzureChatOpenAI(
    openai_api_type = "azure",
    api_version = AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,    
    api_key = AZURE_OPENAI_API_KEY,
    azure_deployment = DEPLOYMENT_NAME_CHAT,
    temperature=0)

# from langchain.document_loaders import WebBaseLoader
# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# docs = loader.load()

# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
summarize_chain = load_summarize_chain(llm, chain_type="stuff")

# chain.run(docs)

#### Main chatbot loop

In [9]:
# Function to handle bot responses

def chat_with_bot(top_k_value):
    retriever = AzureCognitiveSearchRetriever(content_key="content", top_k=top_k_value)
    while True:
        # 0. Get user input
        user_input = input("You: ")
        print("Me:", user_input)
        if user_input.lower() == 'quit':
            print("Exiting conversation.")
            break  # Exit the loop if user inputs 'quit'

        # bot_response = retriever.get_relevant_documents(user_input)
        docs = retriever.get_relevant_documents(user_input)
        bot_response = summarize_chain.run(docs)

        print("Bot:", bot_response)

# Start the conversation
print("Start chatting with the bot ('quit' to exit):")
chat_with_bot(1)

Start chatting with the bot ('quit' to exit):
Me: Oumuamua is what?
Bot: The concise summary of the given information is that various scientific papers and sources have been referenced, discussing topics such as the nature of the interstellar object 'Oumuamua, its non-gravitational acceleration, its cometary fractal aggregate model, and the potential effectiveness of electric sails. These sources also cover topics like X-ray faint isolated neutron stars, nearby stellar groups, extreme ultraviolet sources, and the modeling of 'Oumuamua's light curve.
Me: Oumuamua Considered as an Alien Craft?
Bot: This paper discusses the hypothesis that the interstellar object 'Oumuamua could be an alien spacecraft. The author considers the possibility of both controlled and uncontrolled flight and discusses the challenges and research programs that arise from this hypothesis. The paper highlights the need for a targeted research program to test the alien craft interpretation of 'Oumuamua and suggests 